<a href="https://colab.research.google.com/github/Sparshsing/ai_timelapse/blob/main/time_lapse_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Sparshsing/ai_timelapse.git

Cloning into 'ai_timelapse'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 85 (delta 18), reused 78 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (85/85), 14.16 MiB | 9.08 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [2]:
# change directory to ai_timelapse
%cd ai_timelapse

/content/ai_timelapse


In [5]:
import os
os.getcwd()

'/content/ai_timelapse'

In [6]:
# check GPU (select T4 runtime in settings if error)
!nvidia-smi

Fri Jan 17 06:22:03 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 700.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [9]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.2 MB/s eta 0:00:00


## Generate smooth Time lapse from multiple sequencial images

In [10]:
import os
from pathlib import Path
import numpy as np
import tempfile
import tensorflow as tf
import mediapy
import cv2
from PIL import Image, ExifTags

from natsort import natsorted
import shutil
from tqdm import tqdm
from datetime import datetime
import subprocess
import math
import mediapy as media
import sys
from typing import Generator, Iterable, List, Optional


from eval import interpolator, util
import fix_images

## Config

In [11]:
# Download the pretrained model
import gdown
os.makedirs('pretrained_models/film_net/Style/saved_model', exist_ok=True)

if os.path.exists('pretrained_models/film_net/Style/saved_model/saved_model.pb'):
    print('Model already downloaded')
else:
  folder_url = 'https://drive.google.com/drive/folders/1i9Go1YI2qiFWeT5QtywNFmYAA74bhXWj'
  gdown.download_folder(folder_url, output='pretrained_models/film_net/Style/saved_model', quiet=False)

Retrieving folder contents


Retrieving folder 1NSex_z_Mm13nobMRDyS6JeXLPhW3bREz assets
Retrieving folder 1s9pbFx_bSbinhx5PChJwZqPsyRIlehmZ variables
Processing file 1_oyM-LBAK9o7-bNWf1jG8VvBYeqpmSUr variables.data-00000-of-00001
Processing file 1ceC2kbJs3U1dMMrp4hNIpoHRFxO33SFC variables.index
Processing file 1dT85Z-HyYsiUgIQbOgYFjwWPOw8en1RC keras_metadata.pb
Processing file 1nfi15im3LQvCx84ZRiNcfMuodDkRL_Ei saved_model.pb


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1_oyM-LBAK9o7-bNWf1jG8VvBYeqpmSUr
From (redirected): https://drive.google.com/uc?id=1_oyM-LBAK9o7-bNWf1jG8VvBYeqpmSUr&confirm=t&uuid=6b3d9d53-a046-4c86-860f-c62aa0874661
To: /content/ai_timelapse/pretrained_models/film_net/Style/saved_model/variables/variables.data-00000-of-00001
100%|██████████| 138M/138M [00:02<00:00, 50.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ceC2kbJs3U1dMMrp4hNIpoHRFxO33SFC
To: /content/ai_timelapse/pretrained_models/film_net/Style/saved_model/variables/variables.index
100%|██████████| 1.93k/1.93k [00:00<00:00, 6.95MB/s]
Downloading...
From: https://drive.google.com/uc?id=1dT85Z-HyYsiUgIQbOgYFjwWPOw8en1RC
To: /content/ai_timelapse/pretrained_models/film_net/Style/saved_model/keras_metadata.pb
100%|██████████| 222k/222k [00:00<00:00, 75.1MB/s]
Downloading...
From: https://drive.goo

In [12]:

MAX_DIM = 1080
# set no of interpolated frames between images
MAX_INTERPOLATED_FRAMES = 63  # 1 less than powers of 2
MODEL_PATH = "pretrained_models/film_net/Style/saved_model"

In [13]:
if MAX_INTERPOLATED_FRAMES + 1 not in (2, 4, 8, 16, 32, 64, 128):
    raise ValueError("MAX_INTERPOLATED_FRAMES + 1 must be a power of 2")

In [14]:
input_dir = 'sample_data/shahrukh-khan'
output_dir = 'output_frames/shahrukh-khan'
preprocess_faces = True
equal_intervals = True  # whether to use equal no of frames between images or decide based on date

# applicable when equal_intervals is False
# desired gap between frames in seconds (difference between dates of frames)
# eg. set 3600 to generate 1 frame for every 1 hr gap in input images
min_frame_duration = 3600 * 24

processed_input_dir = tempfile.mkdtemp()
temp_interpolated_frames_dir = tempfile.mkdtemp()

## Plan

In [15]:
# 1. Preprocess images
# 2. for succesive pairs of images:
# 3.     generate interpolated frames
# 4.     add interpolated frames to the output directory
# 5. generate video from output directory

## Utility Functions

In [16]:

# create empty output directory
def create_empty_dir(dir_name):
  if os.path.exists(dir_name):
    shutil.rmtree(dir_name)
  os.makedirs(dir_name)


def resize_and_save(input_path, output_path, size):
  if input_path.endswith(('.png', '.jpg', '.jpeg', '.bmp')):
      img = Image.open(input_path)
      img = img.resize(size, Image.Resampling.LANCZOS)
      img.save(output_path)
  else:
      raise ValueError(f'Unsupported file format: {input_path}')


# prepare input data
def prepare_images(input_dir, processed_input_dir, preprocess_faces, size):
  files = natsorted([f for f in os.listdir(input_dir) if f.endswith(('.png', '.jpg', '.jpeg', '.bmp'))])
  if len(files) < 0:
    raise FileNotFoundError('no images found in input directory')

  for f in tqdm(files, desc='Preparing images'):
    if preprocess_faces:
       fix_images.process_face_image(
            os.path.join(input_dir, f),
            os.path.join(processed_input_dir, f),
            background_color=(255, 255, 255),  # White background
            save_bbox_preview=False
            )
    else:
        resize_and_save(os.path.join(input_dir, f), os.path.join(processed_input_dir, f), size)


def get_new_size(image_path: str, max_dim: int = MAX_DIM) -> np.ndarray:
    """Resize the image so that the maximum dimension is `max_dim`."""
    image = cv2.imread(image_path)
    h, w = image.shape[:2]
    if h > w:
        new_h = max_dim
        new_w = int(w * new_h / h)
    else:
        new_w = max_dim
        new_h = int(h * new_w / w)
    return (new_w, new_h)


def string_to_timestamp(datetime_str):
    # Convert the datetime string to a datetime object
    timestamp = datetime.strptime(datetime_str, "%Y%m%d_%H%M%S")
    return timestamp


def timestamp_to_string(timestamp):
    # Convert the timestamp number to a datetime object
    dt = datetime.fromtimestamp(timestamp)
    # Format the datetime object to the desired string format
    formatted_string = dt.strftime("%Y%m%d_%H%M%S")
    return formatted_string


def choose_evenly_spaced_timestamps(timestamps, k):
    # Ensure we have at least k timestamps
    if k > len(timestamps):
        raise ValueError("k cannot be greater than the number of timestamps")

    # Calculate the interval between timestamps
    n = len(timestamps)
    interval = (n - 1) / (k - 1)

    # Select the timestamps
    chosen_timestamps = []
    for i in range(k):
        index = round(i * interval)
        chosen_timestamps.append(timestamps[index])

    return chosen_timestamps


def save_video(frames, out_path):
    ffmpeg_path = util.get_ffmpeg_path()
    mediapy.set_ffmpeg(ffmpeg_path)
    mediapy.write_video(out_path, frames, fps=30)


def save_frames(frames, output_dir, format='jpg'):
    """
    Save interpolated frames to the specified output directory and return the output paths.
    Args:
        frames: List of image arrays
        output_dir: Directory to save frames
        format: Image format to save (jpg/png)
    Returns:
        list: List of file paths where the frames are saved.
    """
    output_paths = []
    for idx, frame in enumerate(frames):
        output_path = os.path.join(output_dir, f'frame_{idx:06d}.{format}')
        util.write_image(output_path, frame)
        output_paths.append(output_path)
    return output_paths


def write_video_from_images(image_dir, output_path, fps):
    image_files = natsorted([f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg', '.bmp'))])

    if not image_files:
        print("No images found in the directory.")
        return

    # Read the first image to get the dimensions
    first_image_path = os.path.join(image_dir, image_files[0])
    first_frame = cv2.imread(first_image_path)
    height, width, layers = first_frame.shape

    # Initialize the video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Specify the codec
    video_writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    for image_file in image_files:
        image_path = os.path.join(image_dir, image_file)
        frame = cv2.imread(image_path)
        if frame is None:
            print(f"Skipping {image_path}, cannot read image.")
            continue
        video_writer.write(frame)

    video_writer.release()
    print(f"Video saved to: {output_path}")

In [17]:

def interpolate_frames(image_1, image_2, times_to_interpolate, interpolator):
  input_frames = [str(image_1), str(image_2)]

  frames = list(
      util.interpolate_recursively_from_files(
          input_frames, times_to_interpolate, interpolator))
  return frames

## Main

In [18]:
# Preprocess images
files = natsorted([f for f in os.listdir(input_dir) if f.endswith(('.png', '.jpg', '.jpeg', '.bmp'))])
new_size = get_new_size(os.path.join(input_dir, files[0]))
create_empty_dir(processed_input_dir)
prepare_images(input_dir, processed_input_dir, preprocess_faces, new_size)
files = natsorted([f for f in os.listdir(processed_input_dir) if f.endswith(('.png', '.jpg', '.jpeg', '.bmp'))])
len(files)


Preparing images:   0%|          | 0/13 [00:00<?, ?it/s]Downloading data from 'https://github.com/danielgatis/rembg/releases/download/v0.0.0/u2net.onnx' to file '/root/.u2net/u2net.onnx'.

  0%|                                               | 0.00/176M [00:00<?, ?B/s]
  3%|█▏                                    | 5.35M/176M [00:00<00:03, 53.5MB/s]
  6%|██▎                                   | 10.7M/176M [00:00<00:03, 52.2MB/s]
  9%|███▍                                  | 16.0M/176M [00:00<00:03, 52.6MB/s]
 12%|████▌                                 | 21.3M/176M [00:00<00:02, 52.4MB/s]
 15%|█████▊                                | 26.9M/176M [00:00<00:02, 53.6MB/s]
 18%|███████                               | 32.5M/176M [00:00<00:02, 54.5MB/s]
 22%|████████▏                             | 38.0M/176M [00:00<00:02, 51.8MB/s]
 25%|█████████▎                            | 43.2M/176M [00:00<00:02, 50.6MB/s]
 27%|██████████▍                           | 48.2M/176M [00:00<00:02, 50.5MB/s]
 30%|██████

13

In [19]:
if not equal_intervals:
    for file in files:
      timestamp = string_to_timestamp(file.split('.')[0])
      print(f"Filename: {file}, Datetime: {timestamp}")
else:
   for file in files:
      print(f"Filename: {file}")

Filename: 1992_deewana.jpg
Filename: 1993_baazigar.jpg
Filename: 1995_ddlj.jpg
Filename: 1998_kuch_kuch_hota_hai.jpg
Filename: 2001_kbkg.jpg
Filename: 2003_kal_ho_na_ho.jpeg
Filename: 2006_kabh_alvida.jpg
Filename: 2009_billu.jpg
Filename: 2011_don.jpg
Filename: 2014_hny.jpg
Filename: 2017_raees.jpg
Filename: 2019_media.jpg
Filename: 2023_pathan.jpg


In [20]:
# Iterate through pairs of consecutive frames

interpolator_model = interpolator.Interpolator(MODEL_PATH, None)
print('Total frames:', len(files))
create_empty_dir(output_dir)
for i in range(len(files) - 1):
    print('processing frame', i)
    start_filename = files[i]
    end_filename = files[i + 1]

    if not equal_intervals:
        start_time = string_to_timestamp(start_filename.split('.')[0])
        end_time = string_to_timestamp(end_filename.split('.')[0])

    # Calculate times_to_interpolate for the required number of intermediate frames
    if equal_intervals:
          num_frames_needed = MAX_INTERPOLATED_FRAMES
    else:
          # Calculate the time difference in hours
          time_diff = (end_time - start_time).total_seconds()
          num_frames_needed = round(time_diff / min_frame_duration) - 1
          num_frames_needed = min(num_frames_needed, MAX_INTERPOLATED_FRAMES)
    times_to_interpolate = math.ceil(math.log2(num_frames_needed+1))  # n_intermediate = 2^k - 1

    frame_1 = os.path.join(processed_input_dir, start_filename)
    frame_2 = os.path.join(processed_input_dir, end_filename)
    output_frames = interpolate_frames(frame_1, frame_2, times_to_interpolate, interpolator_model)

    create_empty_dir(temp_interpolated_frames_dir)
    save_frames(output_frames, temp_interpolated_frames_dir)
    interpolated_files = natsorted([f for f in os.listdir(temp_interpolated_frames_dir) if f.endswith(('.png', '.jpg', '.jpeg', '.bmp'))])

    if not equal_intervals:
        timestamps = np.linspace(start_time.timestamp(), end_time.timestamp(), len(interpolated_files))
        # find required frames from generated frames (can be extra)
        chosen_timestamps = choose_evenly_spaced_timestamps(timestamps, num_frames_needed+2)

        for filename, timestamp in zip(interpolated_files, timestamps):
            if timestamp not in chosen_timestamps:
                continue
            new_filename = f"{timestamp_to_string(timestamp)}{os.path.splitext(filename)[1]}"
            shutil.copyfile(os.path.join(temp_interpolated_frames_dir, filename), os.path.join(output_dir, new_filename))
    else:
        for interp_index, filename in enumerate(interpolated_files[:-1]):
          new_filename = f"{start_filename.split('.')[0]}_{interp_index}{os.path.splitext(filename)[1]}"
          shutil.copyfile(os.path.join(temp_interpolated_frames_dir, filename), os.path.join(output_dir, new_filename))
          # include end frame in last iteration
          if i == len(files) - 2:
              new_filename = f"{end_filename.split('.')[0]}_0{os.path.splitext(filename)[1]}"
              shutil.copyfile(os.path.join(processed_input_dir, end_filename), os.path.join(output_dir, new_filename))



Total frames: 13
processing frame 0


100%|███████████████████████████████████████████████████████████████| 63/63 [01:49<00:00,  1.74s/it]


processing frame 1


100%|███████████████████████████████████████████████████████████████| 63/63 [01:29<00:00,  1.42s/it]


processing frame 2


...

100%|███████████████████████████████████████████████████████████████| 63/63 [01:28<00:00,  1.41s/it]


processing frame 10


100%|███████████████████████████████████████████████████████████████| 63/63 [01:28<00:00,  1.41s/it]


processing frame 11


100%|███████████████████████████████████████████████████████████████| 63/63 [01:28<00:00,  1.41s/it]


In [21]:

# generate video from output directory
output_path = str(output_dir) + '.mp4'
fps = 24

write_video_from_images(output_dir, output_path, fps)

Video saved to: output_frames/shahrukh-khan.mp4
